In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from sklearn.svm import SVR

In [3]:
import pandas as pd
import numpy as np
import xgboost
# from xgboost import XGBRegressor
# from xgboost import plot_tree #requieres graphviz https://www.graphviz.org/
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
pd.set_option('display.max_columns',None)

In [4]:
df_clean = pd.read_excel("C:/Users/Daniel/OneDrive - Universidad de Antioquia/Analitica y Ciencia de Datos/1/Seminario/Monografia/clean_5.xlsx")

# df_clean.rename(columns={
#     'CURRENT_A_CALC':,
#  'CURRENT_B_CALC':,
#  'CURRENT_C_CALC':,
#  'CURRENT_D_CALC':,
#  'POWER_A':,
#  'POWER_B':,
#  'POWER_C':,
#  'POWER_D':,
#  'TEMPERATURE_A':,
#  'TEMPERATURE_B':,
#  'TEMPERATURE_C':,
#  'TEMPERATURE_D':,
#  'VOLTAGE_A':,
#  'VOLTAGE_B':,
#  'VOLTAGE_C':,
#  'VOLTAGE_D':,
# },inplace=True)
df_clean['ciclo'] = df_clean.groupby(['bateria','fecha_exp'])['fecha_exp'].cumcount()
##df_clean.groupby('bateria')['ciclo'].describe()

lista_metroplus = df_clean[df_clean['lugar']=='metroplus']['fecha_exp'].unique()

In [5]:
variables_ciclo_inicial = [
    'distancia', 'vel_max', 'vel_prom',
    'ACCELERATION_X', 'ACCELERATION_Y', 'ACCELERATION_Z', 'CURRENT_A_CALC',
    'CURRENT_B_CALC', 'CURRENT_C_CALC', 'CURRENT_D_CALC', 'POWER_A',
    'POWER_B', 'POWER_C', 'POWER_D', 'TEMPERATURE_A', 'TEMPERATURE_B',
    'TEMPERATURE_C', 'TEMPERATURE_D', 'VOLTAGE_A', 'VOLTAGE_B', 'VOLTAGE_C',
    'VOLTAGE_D', 'ENV_HUMIDITY', 'ENV_TEMPERATURE','ciclo','bateria'
    ]
variables_ciclo = [x for x in variables_ciclo_inicial if x != 'bateria']

variables_sin_ciclo_inicial = [
    'distancia', 'vel_max', 'vel_prom',
    'ACCELERATION_X', 'ACCELERATION_Y', 'ACCELERATION_Z', 'CURRENT_A_CALC',
    'CURRENT_B_CALC', 'CURRENT_C_CALC', 'CURRENT_D_CALC', 'POWER_A',
    'POWER_B', 'POWER_C', 'POWER_D', 'TEMPERATURE_A', 'TEMPERATURE_B',
    'TEMPERATURE_C', 'TEMPERATURE_D', 'VOLTAGE_A', 'VOLTAGE_B', 'VOLTAGE_C',
    'VOLTAGE_D', 'ENV_HUMIDITY', 'ENV_TEMPERATURE','bateria'
    ]
variables_sin_ciclo = [x for x in variables_sin_ciclo_inicial if x != 'bateria']

In [6]:
X = df_clean[df_clean['fecha_exp'].isin(lista_metroplus)][variables_ciclo_inicial]#[['CURRENT_A_CALC','POWER_A','TEMPERATURE_A','VOLTAGE_A']]
y = df_clean[df_clean['fecha_exp'].isin(lista_metroplus)]['RUL_registro']
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Se escalan los valores del dataset entrenamiento y prueba de las columnas numéricas
X_Min_Max=X.copy(deep=True)
X_Min_Max[X.columns] = scaler.fit_transform(X)
# X_test[num_vars] = scaler.fit_transform(X_test[num_vars])

# Se crean los datasets de entrenamiento y prueba para las variables de entrada y salida
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                        X_Min_Max,
                                        y.values.reshape(-1),
                                        train_size   = 0.8,
                                        random_state = 123,
                                        shuffle      = True,
                                        stratify = X['bateria']
                                    )
X_train.drop('bateria',axis=1,inplace=True)
X_test.drop('bateria',axis=1,inplace=True)

In [7]:
%%time
from sklearn.svm import SVR

svr = SVR(C=100, degree=2, gamma=0.01,kernel='poly',verbose=10)
svr.fit(X_train, y_train)

y_pred = svr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

233.2277122908122
71.85766600414203
Wall time: 2.08 s


In [7]:
%%time
gbr = GradientBoostingRegressor(learning_rate=0.2,max_depth=7,n_estimators=220)
gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_test)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred)*100)

8.448284767452709
99.96307360620786
Wall time: 49.7 s


In [8]:
imp_features = gbr.feature_importances_
# for i in imp_features:
    # print(i.round(3))
df_imp_features = pd.DataFrame({"features":variables_ciclo}).join(pd.DataFrame({"weights":imp_features}))
df_imp_features.sort_values(by=['weights'], ascending=False)

,features,weights
24,ciclo,0.780755
0,distancia,0.067996
2,vel_prom,0.052100
23,ENV_TEMPERATURE,0.015727
1,vel_max,0.012277
22,ENV_HUMIDITY,0.011158
15,TEMPERATURE_B,0.010190
16,TEMPERATURE_C,0.010007
14,TEMPERATURE_A,0.009126
20,VOLTAGE_C,0.007661
